# Safe Working code!

In [1]:

from djitellopy import Tello
import pygame

import numpy as np
import time
from tflite_runtime.interpreter import Interpreter
import cv2

##### TODO: UPDATE THE PARAMETERS IN THIS SECTION #####

label_path = 'dict.txt'
model_path = 'model.tflite'
confidence_threshold = 0.5

##### END OF SECTION #####

# Speed of the drone
S = 60
# Frames per second of the pygame window display
# A low number results in input lag, as input information is processed once per frame
FPS = 120
# Settings for frame size
FRAME_WIDTH = 960
FRAME_HEIGHT = 660

class FrontEnd(object):
    """
        Maintains the Tello display and moves it through the keyboard keys.
        Press escape key to quit.
        Press enter key to take a snapshot.
        The controls are:
        - T: Takeoff
        - L: Land
        - Arrow keys: Forward, backward, left, right
        - A and D: Counter clockwise, clockwise rotations (yaw)
        - W and S: Up, down
    """
    
    def __init__(self):
        # Init pygame
        pygame.init()

        # Create pygame window 
        pygame.display.set_caption("Tello video stream")
        # Set width and height
        self.screen = pygame.display.set_mode([FRAME_WIDTH, FRAME_HEIGHT])

        # Init Tello object that interacts with the Tello drone
        self.tello = Tello()

        # Drone velocities between -100~100
        self.for_back_velocity = 0
        self.left_right_velocity = 0
        self.up_down_velocity = 0
        self.yaw_velocity = 0
        self.speed = 10

        # Initialize snapshot time 
        self.last_snapshot = "N/A"
        self.text_color = (0, 0, 255)
        
        self.send_rc_control = False

        # Import labels 
        with open(label_path, "r") as f:
            self.labels = [line.strip() for line in f.readlines()]
            f.close()

        # Initialize TFLite model and allocate tensors
        self.interpreter = Interpreter(model_path=model_path)
        self.interpreter.allocate_tensors()
        self.input_details, self.output_details = self.interpreter.get_input_details(), self.interpreter.get_output_details()

        # Create update timer
        pygame.time.set_timer(pygame.USEREVENT + 1, 1000 // FPS)

    def run(self):
        self.tello.connect()
        self.tello.set_speed(self.speed)

        # In case streaming is on. This happens when we quit this program without the escape key.
        self.tello.streamoff()
        self.tello.streamon()

        frame_read = self.tello.get_frame_read()

        should_stop = False
        # counter = 0
        while not should_stop:
            for event in pygame.event.get():
                if event.type == pygame.USEREVENT + 1:
                    self.update()
                elif event.type == pygame.QUIT:
                    should_stop = True
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        should_stop = True
                #    else:
                #        self.keydown(event.key)
                #elif event.type == pygame.KEYUP:
                #    self.keyup(event.key)
            
            if frame_read.stopped:
                break
            
            self.screen.fill([0, 0, 0])
            
            # Read and resize image    
            original_shape = np.shape(frame_read.frame)
            input_shape = self.input_details[0]['shape']
            new_image = cv2.resize(frame_read.frame, (input_shape[1], input_shape[2]))

            # counter += 1
            # if counter == 5:
            # Get prediction
            self.interpreter.set_tensor(self.input_details[0]['index'], [new_image])
            self.interpreter.invoke()
            
            boxes = self.interpreter.get_tensor(self.output_details[0]['index']).squeeze()
            classes = self.interpreter.get_tensor(self.output_details[1]['index']).squeeze()
            scores = self.interpreter.get_tensor(self.output_details[2]['index']).squeeze()

            for i in range(len(scores)):
                if scores[i] > confidence_threshold:
                    # Unnormalize boundaries
                    unnormed_coords = boxes[i] * input_shape[1] 
                    start_point = (int(unnormed_coords[1]), int(unnormed_coords[0]))
                    end_point = (int(unnormed_coords[3]), int(unnormed_coords[2]))
                    # Draw bounding box 
                    drawn = cv2.rectangle(new_image, start_point, end_point, color=(0,255,0), thickness=2)
                    # Add label and score
                    img_text = f"{self.labels[int(classes[i])]}: {scores[i]:.3f}"
                    output_label = cv2.putText(new_image, img_text, (int(unnormed_coords[1]), int(unnormed_coords[0])+15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)
                else:
                    break
                
                # counter = 0
            
            frame = frame_read.frame
            new_image = cv2.resize(new_image, (original_shape[1], original_shape[0]))
            # Display battery 
            text = "Battery: {}%".format(self.tello.get_battery())
            cv2.putText(new_image, text, (5, FRAME_HEIGHT - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Display last snapshot timing
            text = "Last snapshot: {}".format(self.last_snapshot)
            cv2.putText(new_image, text, (5, FRAME_HEIGHT - 40), cv2.FONT_HERSHEY_SIMPLEX, 1, self.text_color, 2)
            
            self.new_image = new_image

            new_image = np.rot90(new_image)
            new_image = np.flipud(new_image)
            
            frame = pygame.surfarray.make_surface(new_image)
            self.screen.blit(frame, (0, 0))
            pygame.display.update()

            time.sleep(1 / FPS)

        # Call it always before finishing. To deallocate resources.
        self.tello.end()
    
    def update(self):
        """
            Update routine. Send velocities to Tello.
        """
        if self.send_rc_control:
            self.tello.send_rc_control(self.left_right_velocity, self.for_back_velocity, self.up_down_velocity, self.yaw_velocity)
    
def main():
    frontend = FrontEnd()

    # Run frontend
    frontend.run()

if __name__ == '__main__':
    main()



pygame 2.6.0 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


ImportError: dlopen(/Users/saumil/anaconda3/lib/python3.11/site-packages/tflite_runtime/_pywrap_tensorflow_interpreter_wrapper.so, 0x0002): tried: '/Users/saumil/anaconda3/lib/python3.11/site-packages/tflite_runtime/_pywrap_tensorflow_interpreter_wrapper.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/saumil/anaconda3/lib/python3.11/site-packages/tflite_runtime/_pywrap_tensorflow_interpreter_wrapper.so' (no such file), '/Users/saumil/anaconda3/lib/python3.11/site-packages/tflite_runtime/_pywrap_tensorflow_interpreter_wrapper.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64'))